# DESAFIO FINAL!

In [1]:
# Importando as Bibliotecas

import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

In [2]:
# Lendo os arquivos
csv_market = pd.read_csv('estaticos_market.csv')
csv_portfolio1 = pd.read_csv('portfolio1.csv')
csv_portfolio2 = pd.read_csv('portfolio2.csv')
csv_portfolio3 = pd.read_csv('portfolio3.csv')

# Criando DataFrame para cada um dos arquivos
df_market = pd.DataFrame(csv_market)
df_portfolio1 = pd.DataFrame(csv_portfolio1)
df_portfolio2 = pd.DataFrame(csv_portfolio2)
df_portfolio3 = pd.DataFrame(csv_portfolio3)

In [3]:
df_market.shape

(462298, 182)

In [4]:
df_portfolio1.shape

(555, 182)

In [5]:
df_portfolio2.shape

(566, 2)

### ENTENDENDO OS DADOS

### Tratando os Dados: Modo Geral
#### > Excluindo as colunas com mais de 70% dados faltantes
#### > Pandas Profiling: 
##### - Verificando as colunas que são correlacionadas entre si e excluindo uma das opções;
##### - Excluindo colunas que tem mais de 90% de zeros.

In [6]:
# Criando DataFrame auxiliar: verificar a consistência dos dados
df_market_tamanho = df_market.shape[0]
df_market_consistencia = pd.DataFrame({'COLUNAS': df_market.columns, 'TIPOS': df_market.dtypes, 'FALTANTES': df_market.isna().sum(), 'FALTANTES_PERCENTUAL': round(df_market.isna().sum()/df_market_tamanho,2)})

In [7]:
df_market.shape

(462298, 182)

In [8]:
df_portfolio3.shape

(265, 2)

In [9]:
# Acrescentando uma coluna no DataFrame para informar se algum atrituto tem mais de 70% de dados faltantes
df_market_consistencia['TemMaisDe70%Faltantes'] = ['sim' if x > 0.7 else 'nao' for x in df_market_consistencia['FALTANTES_PERCENTUAL']]

In [10]:
df_market_consistencia.head()

,COLUNAS,TIPOS,FALTANTES,FALTANTES_PERCENTUAL,TemMaisDe70%Faltantes
Unnamed: 0,Unnamed: 0,int64,0,0.0,nao
id,id,object,0,0.0,nao
fl_matriz,fl_matriz,bool,0,0.0,nao
de_natureza_juridica,de_natureza_juridica,object,0,0.0,nao
sg_uf,sg_uf,object,0,0.0,nao


In [11]:
# Excluindo as colunas que tem mais de 70% de dados faltantes
manter_semFaltantes = list(df_market_consistencia[df_market_consistencia['TemMaisDe70%Faltantes'] == 'nao']['COLUNAS'])
df_market = df_market[manter_semFaltantes]

In [12]:
# Verificando quantas colunas excluimos
# Descartamos 129 variáveis!
df_market.shape

(462298, 53)

In [13]:
# Selecionando e excluindo colunas com as seguintes caracteristicas:
# constantes, com mais de 90% de valor 'zero'
lista_excluir_consEzerosE50faltantes = ['vl_total_veiculos_pesados_grupo', 'vl_total_veiculos_leves_grupo']
df_market.drop(columns = lista_excluir_consEzerosE50faltantes, inplace=True)

In [14]:
df_market.drop(columns = 'dt_situacao', inplace = True)
df_market.shape

(462298, 50)

In [15]:
# Verificando a quantidade de dados faltantes de cada columa
# Resultado: temos algumas colunas com dados faltantes! Precisamos imputar dados.
df_market.isna().sum()

Unnamed: 0                                    0
id                                            0
fl_matriz                                     0
de_natureza_juridica                          0
sg_uf                                         0
natureza_juridica_macro                       0
de_ramo                                       0
setor                                      1927
idade_empresa_anos                            0
idade_emp_cat                                 0
fl_me                                         0
fl_sa                                         0
fl_epp                                        0
fl_mei                                        0
fl_ltda                                       0
fl_st_especial                                0
fl_email                                      0
fl_telefone                                   0
fl_rm                                         0
nm_divisao                                 1927
nm_segmento                             

In [16]:
df_market.shape

(462298, 50)

### SEPARANDO AS COLUNAS POR TIPOS!

In [17]:
# Verificando os tipos dos dados presente neste DataFrame
df_market.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('bool'), dtype('float64')],
      dtype=object)

In [18]:
df_market_tipos_aux = pd.DataFrame({'colunas': df_market.columns, 'tipos':df_market.dtypes})
df_market_tipos_aux.head()

,colunas,tipos
Unnamed: 0,Unnamed: 0,int64
id,id,object
fl_matriz,fl_matriz,bool
de_natureza_juridica,de_natureza_juridica,object
sg_uf,sg_uf,object


In [19]:
# Separando os tipos em listas!
lista_dadosNumericos = list(df_market_tipos_aux[(df_market_tipos_aux['tipos'] == 'int64') | (df_market_tipos_aux['tipos'] == 'float64')]['colunas'])
lista_dadosBool = list(df_market_tipos_aux[(df_market_tipos_aux['tipos'] == 'bool')]['colunas'])
lista_dadosObject = list(df_market_tipos_aux[(df_market_tipos_aux['tipos'] == 'object')]['colunas'])

In [20]:
# Criando DataFrames de acordo com as listas acima!
df_market_dadosNumericos = df_market[lista_dadosNumericos]
df_market_dadosBool = df_market[lista_dadosBool]
df_market_dadosObject = df_market[lista_dadosObject]

# (df_market_todoNumerico)! É o DataFrame todo numérico para conseguirmos trabalhar!

In [21]:
df_market_dadosObject.columns

Index(['id', 'de_natureza_juridica', 'sg_uf', 'natureza_juridica_macro',
       'de_ramo', 'setor', 'idade_emp_cat', 'fl_rm', 'nm_divisao',
       'nm_segmento', 'fl_spa', 'fl_antt', 'fl_veiculo', 'fl_optante_simples',
       'fl_optante_simei', 'sg_uf_matriz', 'de_saude_tributaria',
       'de_saude_rescencia', 'de_nivel_atividade', 'fl_simples_irregular',
       'nm_meso_regiao', 'nm_micro_regiao', 'fl_passivel_iss',
       'de_faixa_faturamento_estimado', 'de_faixa_faturamento_estimado_grupo'],
      dtype='object')

### Trabalhando com DADOS "bool"

In [22]:
# Não temos nenhum dado faltante neste DataFrame!
df_market_dadosBool.isna().sum()

fl_matriz         0
fl_me             0
fl_sa             0
fl_epp            0
fl_mei            0
fl_ltda           0
fl_st_especial    0
fl_email          0
fl_telefone       0
dtype: int64

In [23]:
# Transformando os dados "bool" em "int"
df_market_dadosBool_transformado = df_market_dadosBool.astype(int)
df_market_dadosBool_transformado.head()

,fl_matriz,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone
0,1,0,0,0,0,0,0,1,1
1,1,0,0,0,1,0,0,1,1
2,1,0,0,0,1,0,0,0,1
3,1,0,0,0,0,0,0,1,1
4,1,0,0,0,0,0,0,1,1


In [24]:
# Adicionando (df_market_dadosBool_transformado) ao (df_market_todoNumerico)
df_market_todoNumerico = df_market_dadosBool_transformado

In [25]:
# Verificando se o (df_market_dadosBool_transformado) foi adicionado corretamente em (df_market_todoNumerico)
df_market_todoNumerico.shape

(462298, 9)

In [26]:
# Verificando a quantidade de dados faltantes no (df_market_todoNumerico). Continua zerado. 
# RESULTADO: Nenhum dado faltante! ÓTIMO, eba :D
df_market_todoNumerico.isna().sum()

fl_matriz         0
fl_me             0
fl_sa             0
fl_epp            0
fl_mei            0
fl_ltda           0
fl_st_especial    0
fl_email          0
fl_telefone       0
dtype: int64

### Trabalhando com DADOS NUMÉRICOS : "int64"e "float64"

In [27]:
df_market_todoNumerico.columns

Index(['fl_matriz', 'fl_me', 'fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda',
       'fl_st_especial', 'fl_email', 'fl_telefone'],
      dtype='object')

In [28]:
df_market_dadosNumericos.isna().sum()

Unnamed: 0                                    0
idade_empresa_anos                            0
nu_meses_rescencia                        45276
empsetorcensitariofaixarendapopulacao    143829
qt_socios                                115091
qt_socios_pf                             115091
qt_socios_pj                             115091
idade_media_socios                       151602
idade_maxima_socios                      151602
idade_minima_socios                      151602
qt_socios_st_regular                     154917
qt_socios_masculino                      267183
qt_socios_feminino                       317269
vl_faturamento_estimado_aux               27513
vl_faturamento_estimado_grupo_aux         27513
qt_filiais                                    0
dtype: int64

In [29]:
df_market_dadosNumericos.drop(columns = 'Unnamed: 0', inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [30]:
df_market_dadosNumericos_CONSISTENCIA = pd.DataFrame({'colunas': df_market_dadosNumericos.columns, 'missing_Quantidade': df_market_dadosNumericos.isna().sum()})

In [31]:
df_market_dadosNumericos_CONSISTENCIA.head()

,colunas,missing_Quantidade
idade_empresa_anos,idade_empresa_anos,0
nu_meses_rescencia,nu_meses_rescencia,45276
empsetorcensitariofaixarendapopulacao,empsetorcensitariofaixarendapopulacao,143829
qt_socios,qt_socios,115091
qt_socios_pf,qt_socios_pf,115091


In [32]:
lista_market_dadosNumericos_SEM_Faltantes = list(df_market_dadosNumericos_CONSISTENCIA[df_market_dadosNumericos_CONSISTENCIA['missing_Quantidade'] == 0]['colunas'])

In [33]:
lista_market_dadosNumericos_SEM_Faltantes

['idade_empresa_anos', 'qt_filiais']

In [34]:
df_market_dadosNumericos_SEM_Faltantes = pd.DataFrame(df_market_dadosNumericos[lista_market_dadosNumericos_SEM_Faltantes])

In [35]:
df_market_dadosNumericos_SEM_Faltantes.head()

,idade_empresa_anos,qt_filiais
0,14.457534,0
1,1.463014,0
2,7.093151,0
3,6.512329,0
4,3.200000,0


In [36]:
lista_market_dadosNumericos_COM_Faltantes = list(df_market_dadosNumericos_CONSISTENCIA[df_market_dadosNumericos_CONSISTENCIA['missing_Quantidade'] > 0]['colunas'])

In [37]:
lista_market_dadosNumericos_COM_Faltantes

['nu_meses_rescencia',
 'empsetorcensitariofaixarendapopulacao',
 'qt_socios',
 'qt_socios_pf',
 'qt_socios_pj',
 'idade_media_socios',
 'idade_maxima_socios',
 'idade_minima_socios',
 'qt_socios_st_regular',
 'qt_socios_masculino',
 'qt_socios_feminino',
 'vl_faturamento_estimado_aux',
 'vl_faturamento_estimado_grupo_aux']

In [38]:
df_market_dadosNumericos_COM_Faltantes = pd.DataFrame(df_market_dadosNumericos[lista_market_dadosNumericos_COM_Faltantes])

In [39]:
df_market_dadosNumericos_COM_Faltantes.shape

(462298, 13)

In [40]:
df_market_dadosNumericos.isna().sum()

idade_empresa_anos                            0
nu_meses_rescencia                        45276
empsetorcensitariofaixarendapopulacao    143829
qt_socios                                115091
qt_socios_pf                             115091
qt_socios_pj                             115091
idade_media_socios                       151602
idade_maxima_socios                      151602
idade_minima_socios                      151602
qt_socios_st_regular                     154917
qt_socios_masculino                      267183
qt_socios_feminino                       317269
vl_faturamento_estimado_aux               27513
vl_faturamento_estimado_grupo_aux         27513
qt_filiais                                    0
dtype: int64

In [41]:
# Substituindo todos os dados numéricos faltantes por "999999999"
# Dessa forma vamos conseguir usar algum algoritmo de regressão e mesmo assim vamos saber que é NaN
df_market_dadosNumericos = df_market_dadosNumericos.fillna(999999999)

In [42]:
# Testando para ver se funcionou.
# Verificando a quantidade de dados faltantes nas colunas numéricas. Total = 0, uhuuuL
df_market_dadosNumericos.isna().sum()

idade_empresa_anos                       0
nu_meses_rescencia                       0
empsetorcensitariofaixarendapopulacao    0
qt_socios                                0
qt_socios_pf                             0
qt_socios_pj                             0
idade_media_socios                       0
idade_maxima_socios                      0
idade_minima_socios                      0
qt_socios_st_regular                     0
qt_socios_masculino                      0
qt_socios_feminino                       0
vl_faturamento_estimado_aux              0
vl_faturamento_estimado_grupo_aux        0
qt_filiais                               0
dtype: int64

In [43]:
# Adicionando (df_market_dadosNumericos) ao (df_market_todoNumerico)
#df_market_todoNumerico = df_market_todoNumerico.join(df_market_dadosNumericos)

In [44]:
# Verificando o número de linhas e colunas no DataFrame (df_market_todoNumerico)
#df_market_todoNumerico.shape

In [45]:
df_market_todoNumerico.shape

(462298, 9)

In [46]:
df_market_todoNumerico_COPIA_dadosNumericos = df_market_todoNumerico

### >>> IMPUTANDO VALORES: 'int64' e 'float64' <<<

In [47]:
from sklearn.linear_model import LinearRegression

In [48]:
reg = LinearRegression()

In [49]:
df_market_dadosNumerico_transformados = df_market_dadosNumericos

In [50]:
df_market_dadosNumericos.shape

(462298, 15)

In [51]:
df_market_dadosNumerico_transformados.shape

(462298, 15)

In [52]:
# Separando as Bases de TREINO e TESTE

for x in lista_market_dadosNumericos_COM_Faltantes:
    
    print('Começando a coluna: ',x)
    
    df_total = df_market_todoNumerico_COPIA_dadosNumericos.join(df_market_dadosNumericos[x])

    df_market_dadosNumericos_TREINO = pd.DataFrame(df_total[df_total[x] != 999999999])
    df_market_dadosNumericos_TESTE = pd.DataFrame(df_total[df_total[x] == 999999999])
    
    # Definindo o TREINO
    Y_TREINO_coluna = df_market_dadosNumericos_TREINO[x]
    X_TREINO_coluna = df_market_dadosNumericos_TREINO.drop(columns = x)
 
    # Definindo TESTE
    X_TESTE_coluna = df_market_dadosNumericos_TESTE.drop(columns = x)

    reg.fit(X_TREINO_coluna, Y_TREINO_coluna)
    Y_PREDITO = reg.predict(X_TESTE_coluna)
    
    # Substituindo '999999999' pelos valores calculado' no (df_market_dadosNumerico_transformados)
    # Porque (df_market_dadosNumerico_transformados)
    df_market_dadosNumerico_transformados[x].replace({999999999: Y_PREDITO}, inplace = True)


Começando a coluna:  nu_meses_rescencia
Começando a coluna:  empsetorcensitariofaixarendapopulacao
Começando a coluna:  qt_socios
Começando a coluna:  qt_socios_pf
Começando a coluna:  qt_socios_pj
Começando a coluna:  idade_media_socios
Começando a coluna:  idade_maxima_socios
Começando a coluna:  idade_minima_socios
Começando a coluna:  qt_socios_st_regular
Começando a coluna:  qt_socios_masculino
Começando a coluna:  qt_socios_feminino
Começando a coluna:  vl_faturamento_estimado_aux
Começando a coluna:  vl_faturamento_estimado_grupo_aux


In [53]:
df_market_todoNumerico = df_market_todoNumerico.join(df_market_dadosNumerico_transformados)

In [54]:
df_market_todoNumerico.shape

(462298, 24)

In [55]:
df_market_todoNumerico.columns

Index(['fl_matriz', 'fl_me', 'fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda',
       'fl_st_especial', 'fl_email', 'fl_telefone', 'idade_empresa_anos',
       'nu_meses_rescencia', 'empsetorcensitariofaixarendapopulacao',
       'qt_socios', 'qt_socios_pf', 'qt_socios_pj', 'idade_media_socios',
       'idade_maxima_socios', 'idade_minima_socios', 'qt_socios_st_regular',
       'qt_socios_masculino', 'qt_socios_feminino',
       'vl_faturamento_estimado_aux', 'vl_faturamento_estimado_grupo_aux',
       'qt_filiais'],
      dtype='object')

In [56]:
# Já temos no nosso DataFrame todos os valores numéricos do tipo: 'bool', 'int64, 'float64'
# Agora vamos mexer nos dados do tipo: object!

### >>> IMPUTANDO VALORES: 'object' <<<

In [57]:
# Lembrando que:
# Nosso DataFrame resultado (até o momento) é: df_market_todoNumerico
# O DataFrame (df_market_dadosObject) é o DataFrame com todos os dados do tipo "object"
# A ideia é usar um algoritmo de Classificação (ex: Árvore de Decisão) para substituir os dados faltantes
# Estamos assumindo que os dados faltantes serão representados por: "souNulo"

In [58]:
df_market_todoNumerico.columns

Index(['fl_matriz', 'fl_me', 'fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda',
       'fl_st_especial', 'fl_email', 'fl_telefone', 'idade_empresa_anos',
       'nu_meses_rescencia', 'empsetorcensitariofaixarendapopulacao',
       'qt_socios', 'qt_socios_pf', 'qt_socios_pj', 'idade_media_socios',
       'idade_maxima_socios', 'idade_minima_socios', 'qt_socios_st_regular',
       'qt_socios_masculino', 'qt_socios_feminino',
       'vl_faturamento_estimado_aux', 'vl_faturamento_estimado_grupo_aux',
       'qt_filiais'],
      dtype='object')

In [59]:
df_market_dadosObject.isna().sum()

id                                         0
de_natureza_juridica                       0
sg_uf                                      0
natureza_juridica_macro                    0
de_ramo                                    0
setor                                   1927
idade_emp_cat                              0
fl_rm                                      0
nm_divisao                              1927
nm_segmento                             1927
fl_spa                                  1927
fl_antt                                 1927
fl_veiculo                              1927
fl_optante_simples                     82713
fl_optante_simei                       82713
sg_uf_matriz                            1939
de_saude_tributaria                    14851
de_saude_rescencia                     14851
de_nivel_atividade                     11168
fl_simples_irregular                    1927
nm_meso_regiao                         58698
nm_micro_regiao                        58698
fl_passive

In [60]:
# Fazendo um DataFrame auxiliar para ajudar a verificar a consistencia destes dados!
df_market_dadosObject_CONSISTENCIA = pd.DataFrame({'colunas': df_market_dadosObject.columns, 'missing_Quantidade': df_market_dadosObject.isna().sum()})

In [61]:
# Lista com as variáveis que tem algum dado faltante
lista_dadosObject_COM_dadosFaltantes = list(df_market_dadosObject_CONSISTENCIA[df_market_dadosObject_CONSISTENCIA['missing_Quantidade'] > 0]['colunas'])

In [62]:
# Lista com as variáveis com nenhum dado faltante
lista_dadosObject_SEM_dadosFaltantes = list(df_market_dadosObject_CONSISTENCIA[df_market_dadosObject_CONSISTENCIA['missing_Quantidade'] == 0]['colunas'])

In [63]:
# Vamos isolar "id"
# Ou seja, salvar em uma variável e excluir do DataFrame (df_market_dadosObject)
# Porque esta coluna não vai fazer parte de nenhum aprendizado, mas precisamos os valores dos "id"s das empresas.
armazenando_ID = df_market_dadosObject['id']

In [64]:
#lista_dadosObject_aux = list(df_market_dadosObject.columns)
#lista_dadosObject_aux.remove('id')
#lista_dadosObject_aux

In [65]:
# Substituindo todos os dados numéricos faltantes por 'souNulo'
# Dessa forma vamos conseguir usar algum algoritmo de classificacao e mesmo assim vamos saber que é NaN
df_market_dadosObject = df_market_dadosObject.fillna('souNulo')

In [66]:
df_market_dadosObject.isna().sum()

id                                     0
de_natureza_juridica                   0
sg_uf                                  0
natureza_juridica_macro                0
de_ramo                                0
setor                                  0
idade_emp_cat                          0
fl_rm                                  0
nm_divisao                             0
nm_segmento                            0
fl_spa                                 0
fl_antt                                0
fl_veiculo                             0
fl_optante_simples                     0
fl_optante_simei                       0
sg_uf_matriz                           0
de_saude_tributaria                    0
de_saude_rescencia                     0
de_nivel_atividade                     0
fl_simples_irregular                   0
nm_meso_regiao                         0
nm_micro_regiao                        0
fl_passivel_iss                        0
de_faixa_faturamento_estimado          0
de_faixa_faturam

In [67]:
df_parcial_dadosObject_SEM_dadosFaltantes = pd.DataFrame(df_market_dadosObject['id'])
lista_dadosObject_SEM_dadosFaltantes

['id',
 'de_natureza_juridica',
 'sg_uf',
 'natureza_juridica_macro',
 'de_ramo',
 'idade_emp_cat',
 'fl_rm']

In [68]:
lista_dadosObject_SEM_dadosFaltantes.remove('id')

In [69]:
lista_dadosObject_SEM_dadosFaltantes

['de_natureza_juridica',
 'sg_uf',
 'natureza_juridica_macro',
 'de_ramo',
 'idade_emp_cat',
 'fl_rm']

In [70]:
# Transformando as colunas em dados numéricos e salvando num novo DataFrame
# SOMENTE AS COLUNAS DO TIPO "OBJECT" SEM DADOS FALTANTES!

for x in lista_dadosObject_SEM_dadosFaltantes:
    df_market_aux = pd.get_dummies(df_market_dadosObject[x], prefix = x)
    df_parcial_dadosObject_SEM_dadosFaltantes = df_parcial_dadosObject_SEM_dadosFaltantes.join(df_market_aux)

In [71]:
# Verificando o formato do DataFrame que tivemos como resultado!
df_parcial_dadosObject_SEM_dadosFaltantes.shape

(462298, 122)

In [72]:
# Precisamos excluir o "id" do DataFrame que vamos utilizar para realizar a classificacao
# Porque nao queremos que "id" interfira.
df_parcial_dadosObject_SEM_dadosFaltantes.drop(columns = 'id', inplace = True)

In [73]:
# Testando de "id" sumiu mesmo!
# Resultado: sim, sumiu!
# df_parcial_dadosObject['id']

In [74]:
# Primeiro vamos fazer uma cópia do DataFrame (df_market_todoNumerico)
# Vamos adicionar o DataFrame resultado depois de inserir todos os dados faltantes do tipo "bool"
# Adicionando o DataFrame (df_parcial_dadosObject) ao DataFrame (df_market_todoNumerico)
# OBS: a coluna "id"até o momento naão esta fazendo parte de nenhum DataFrame!
df_market_todoNumerico_COPIA_Object = df_market_todoNumerico
df_market_todoNumerico_COPIA_Object = df_market_todoNumerico_COPIA_Object.join(df_parcial_dadosObject_SEM_dadosFaltantes)

In [75]:
df_market_todoNumerico_COPIA_Object.shape

(462298, 145)

### >>> Informações! <<<

In [76]:
# Vamos usar o DataFrame (df_market_todoNumerico_COPIA) para realizar as operações de descobrir os valores faltantes!
# Este DataFrame está completo.
# Os valores nulos foram substituidos por: 'souNulo'

In [77]:
from sklearn import tree

In [78]:
clf = tree.DecisionTreeClassifier()

In [79]:
# (lista_dadosObject_COM_dadosFaltantes) : contém todas as colunas do tipo "bool"que tem dados faltantes
lista_dadosObject_COM_dadosFaltantes

['setor',
 'nm_divisao',
 'nm_segmento',
 'fl_spa',
 'fl_antt',
 'fl_veiculo',
 'fl_optante_simples',
 'fl_optante_simei',
 'sg_uf_matriz',
 'de_saude_tributaria',
 'de_saude_rescencia',
 'de_nivel_atividade',
 'fl_simples_irregular',
 'nm_meso_regiao',
 'nm_micro_regiao',
 'fl_passivel_iss',
 'de_faixa_faturamento_estimado',
 'de_faixa_faturamento_estimado_grupo']

In [80]:
# Criando uma cópia do DataFrame (df_market_dadosObject)
df_market_dadosObject_COPIA = df_market_dadosObject

In [81]:
from sklearn import preprocessing

In [82]:
le = preprocessing.LabelEncoder()

In [83]:
# Separando as Bases de TREINO e TESTE

for x in lista_dadosObject_COM_dadosFaltantes:
    print('Começando a coluna: ',x)
    
    df_total = df_market_todoNumerico_COPIA_Object.join(df_market_dadosObject_COPIA[x])

    df_market_TN_treino = df_total[df_total[x] != 'souNulo']
    df_market_TN_teste = df_total[df_total[x] == 'souNulo']
    
    # Definindo o TREINO
    Y_TREINO_coluna = le.fit_transform(df_market_TN_treino[x])
    X_TREINO_coluna = df_market_TN_treino.drop(columns = x)

    # Definindo TESTE
    X_TESTE_coluna = df_market_TN_teste.drop(columns = x)
    
    clf.fit(X_TREINO_coluna, Y_TREINO_coluna)
    Y_PREDITO = clf.predict(X_TESTE_coluna)
    Y_PREDITO_RESULTADO = le.inverse_transform(Y_PREDITO)
    
    # Substituindo 'souNulo' pelos valores calculado'
    df_market_dadosObject_COPIA[x].replace({'souNulo': Y_PREDITO_RESULTADO}, inplace = True)


Começando a coluna:  setor
Começando a coluna:  nm_divisao
Começando a coluna:  nm_segmento
Começando a coluna:  fl_spa
Começando a coluna:  fl_antt
Começando a coluna:  fl_veiculo
Começando a coluna:  fl_optante_simples
Começando a coluna:  fl_optante_simei
Começando a coluna:  sg_uf_matriz
Começando a coluna:  de_saude_tributaria
Começando a coluna:  de_saude_rescencia
Começando a coluna:  de_nivel_atividade
Começando a coluna:  fl_simples_irregular
Começando a coluna:  nm_meso_regiao
Começando a coluna:  nm_micro_regiao
Começando a coluna:  fl_passivel_iss
Começando a coluna:  de_faixa_faturamento_estimado
Começando a coluna:  de_faixa_faturamento_estimado_grupo


In [84]:
# Agora (df_market_dadosObject_COPIA) está completa!
# Vamos transformar este DataFrame em um dataFrame numérico com o (get_dummies)

In [85]:
df_market_dadosObject_COPIA.columns

Index(['id', 'de_natureza_juridica', 'sg_uf', 'natureza_juridica_macro',
       'de_ramo', 'setor', 'idade_emp_cat', 'fl_rm', 'nm_divisao',
       'nm_segmento', 'fl_spa', 'fl_antt', 'fl_veiculo', 'fl_optante_simples',
       'fl_optante_simei', 'sg_uf_matriz', 'de_saude_tributaria',
       'de_saude_rescencia', 'de_nivel_atividade', 'fl_simples_irregular',
       'nm_meso_regiao', 'nm_micro_regiao', 'fl_passivel_iss',
       'de_faixa_faturamento_estimado', 'de_faixa_faturamento_estimado_grupo'],
      dtype='object')

In [86]:
df_market_dadosObject_COMPLETO = df_market_dadosObject_COPIA

In [87]:
df_market_dadosObject_COMPLETO.drop(columns = 'id', inplace = True)

In [88]:
df_market_dadosObject_transformado = pd.get_dummies(df_market_dadosObject)

In [89]:
df_market_dadosObject_transformado.shape

(462298, 398)

In [90]:
df_market_dadosObject_transformado.columns

Index(['fl_spa', 'fl_antt', 'fl_veiculo', 'fl_optante_simples',
       'fl_optante_simei', 'fl_simples_irregular', 'fl_passivel_iss',
       'de_natureza_juridica_ASSOCIACAO PRIVADA',
       'de_natureza_juridica_AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL',
       'de_natureza_juridica_AUTARQUIA FEDERAL',
       ...
       'de_faixa_faturamento_estimado_grupo_ATE R$ 81.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 1.500.000,01 A R$ 4.800.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 10.000.000,01 A R$ 30.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 100.000.000,01 A R$ 300.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 30.000.000,01 A R$ 100.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 300.000.000,01 A R$ 500.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 360.000,01 A R$ 1.500.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 4.800.000,01 A R$ 10.000.000,00',
       'de_faixa_faturament

In [91]:
# Verificando os dados que são do tipo 'bool'
lista_auxiliar_bool = list(df_market_dadosObject_transformado.select_dtypes(include ='bool'))
df_auxiliar_bool = df_market_dadosObject_transformado[lista_auxiliar_bool]
df_auxiliar_bool.astype(int)

,fl_spa,fl_antt,fl_veiculo,fl_optante_simples,fl_optante_simei,fl_simples_irregular,fl_passivel_iss
0,0,0,0,1,0,0,1
1,0,0,0,1,0,0,1
2,0,0,0,1,1,0,1
3,0,0,0,1,0,0,1
4,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...
462293,0,0,0,1,1,0,0
462294,0,0,0,0,0,0,1
462295,0,0,0,0,0,0,1
462296,0,0,0,0,0,0,0


In [92]:
df_market_dadosObject_transformado.drop(columns = lista_auxiliar_bool, inplace = True)

In [93]:
df_market_dadosObject_transformado.columns

Index(['de_natureza_juridica_ASSOCIACAO PRIVADA',
       'de_natureza_juridica_AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL',
       'de_natureza_juridica_AUTARQUIA FEDERAL',
       'de_natureza_juridica_AUTARQUIA MUNICIPAL',
       'de_natureza_juridica_CANDIDATO A CARGO POLITICO ELETIVO',
       'de_natureza_juridica_CLUBE FUNDO DE INVESTIMENTO',
       'de_natureza_juridica_COMUNIDADE INDIGENA',
       'de_natureza_juridica_CONDOMINIO EDILICIO',
       'de_natureza_juridica_CONSORCIO DE EMPREGADORES',
       'de_natureza_juridica_CONSORCIO DE SOCIEDADES',
       ...
       'de_faixa_faturamento_estimado_grupo_ATE R$ 81.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 1.500.000,01 A R$ 4.800.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 10.000.000,01 A R$ 30.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 100.000.000,01 A R$ 300.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 30.000.000,01 A R$ 100.000.000,00',
       'de_faixa_fatura

In [94]:
df_market_dadosObject_transformado = df_market_dadosObject_transformado.join(df_auxiliar_bool)

In [95]:
df_market_dadosObject_transformado.shape

(462298, 398)

In [96]:
# Agora todos os dados estão completos!
# Vamos juntar todos os DataFrames!

## JUNTANDO TODOS OS DATAFRAMES TRATADOS!

In [97]:
# Nosso DataFrame final vai ser o (df_dadosTratados)

In [98]:
# Primeiro estamos adicionando os valores do ID
df_dadosTratados = pd.DataFrame(armazenando_ID)

In [99]:
df_dadosTratados.shape

(462298, 1)

In [100]:
# Fazendo JOIN (df_dadosTratados) com os dados do tipo: "bool"
df_dadosTratados = df_dadosTratados.join(df_market_dadosBool_transformado)

In [101]:
# Fazendo JOIN (df_dadosTratados) com os dados do tipo: "int64" e "float64"
df_dadosTratados = df_dadosTratados.join(df_market_dadosNumerico_transformados)

In [102]:
# Fazendo JOIN (df_dadosTratados) com os dados do tipo: object (df_market_dadosObject_transformado)
df_dadosTratados = df_dadosTratados.join(df_market_dadosObject_transformado)

In [103]:
df_dadosTratados.shape

(462298, 423)

In [104]:
df_dadosTratados.columns

Index(['id', 'fl_matriz', 'fl_me', 'fl_sa', 'fl_epp', 'fl_mei', 'fl_ltda',
       'fl_st_especial', 'fl_email', 'fl_telefone',
       ...
       'de_faixa_faturamento_estimado_grupo_DE R$ 4.800.000,01 A R$ 10.000.000,00',
       'de_faixa_faturamento_estimado_grupo_DE R$ 500.000.000,01 A 1 BILHAO DE REAIS',
       'de_faixa_faturamento_estimado_grupo_DE R$ 81.000,01 A R$ 360.000,00',
       'fl_spa', 'fl_antt', 'fl_veiculo', 'fl_optante_simples',
       'fl_optante_simei', 'fl_simples_irregular', 'fl_passivel_iss'],
      dtype='object', length=423)

## Salvando o DataFrame numérico num arquivo para facilitar a vida!

In [105]:
df_dadosTratados.to_csv('dataframe_completo.csv')